## Esercizio

Estendere il linguaggio MiniCaml e l’implementazione dell’interprete con il nuovo costrutto condizionale `On G1 -> E1 | G2 -> E2`. `G1` e `G2` sono le guardie, e vengono valutate a dei booleani. In base ai loro valori, valuta l'espressione `E1` o `E2` corrispondente alla guardia `true`.

Semantica operazionale big step (non deterministica):

$$
\frac{G_1 \rightarrow_{bs} \mbox{true} \quad E_1 \rightarrow_{bs} v}{\mbox{on } G_1 ; E_1 | G_2 ; E_2 \rightarrow_{bs} v}
$$

$$
\frac{G_2 \rightarrow_{bs} \mbox{true} \quad E_2 \rightarrow_{bs} v}{\mbox{on } G_1 ; E_1 | G_2 ; E_2 \rightarrow_{bs} v}
$$

Per renderla deterministica (come scegliere la strategia call-by-value o call-by-name nel lambda calcolo), sostituiamo la seconda regola con:

$$
\frac{G_1 \rightarrow_{bs} \mbox{false} \quad G_2 \rightarrow_{bs} \mbox{true} \quad E_2 \rightarrow_{bs} v}{\mbox{on } G_1 ; E_1 | G_2 ; E_2 \rightarrow_{bs} v}
$$

che obbliga la prima guardia ad essere false. In questo modo, se entrambe le guardie sono true si può applicare solo la prima regola, che valuta $E_1$.


Semantica operazionale small step (non deterministica):

$$
\frac{G_1 \rightarrow_{ss} \mbox{true}}{\mbox{on } G_1 ; E_1 | G_2 ; E_2 \rightarrow_{ss} E_1}
$$

$$
\frac{G_2 \rightarrow_{ss} \mbox{true}}{\mbox{on } G_1 ; E_1 | G_2 ; E_2 \rightarrow_{ss} E_2}
$$

Classifichiamo gli oggetti di interesse in tre categorie:
- Espressioni (o sintassi) MiniCaml (tipo `exp`)
- Valori MiniCaml (tipo `evT`)
- Valori OCaml

Il costrutto `On` è un'espressione. Non ci servono invece nuovi valori MiniCaml.

### Espressioni

In [1]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* Costrutto On *)
    | On of exp * exp * exp * exp
    (* ForEach
    | ForEach of exp list * exp
    *)
;;

type ide = string


type tname = TInt | TBool | TString | TClosure | TRecClosure | TUnBound


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | On of exp * exp * exp * exp


### Tipi esprimibili

In [2]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int  
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env
    | RecClosure of ide * ide * exp * evT env
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


### Typechecking

In [3]:
(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )
    

val getType : evT -> tname = <fun>


val typecheck : tname * evT -> bool = <fun>


### Interprete

In [4]:
(* Errori a runtime *)
exception RuntimeError of string

(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

exception RuntimeError of string


val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


...

val less_than : evT * evT -> evT = <fun>


val greater_than : evT * evT -> evT = <fun>


val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


In [5]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (* On *)
    | On(eg1, e1, eg2, e2) ->
        (* eg1, e1, eg2, e2 : exp sono ESPRESSIONI MiniCaml *)
        let vg1: evT = eval eg1 s in
        let vg2: evT = eval eg2 s in
        (* vg1, vg2 sono VALORI MiniCaml *)
        (match vg1, vg2 with
        | Bool(true), Bool(_) -> eval e1 s
        | Bool(false), Bool(true) -> eval e2 s
        | Bool(false), Bool(false) -> raise ( RuntimeError "Both guards are false")
        | _, _ -> raise ( RuntimeError "Wrong type")
        )
        (*
        (match typecheck(TBool, vg1), typecheck(TBool, vg2), vg1, vg2 with
        | true, true, Bool(true), Bool(_) -> ...
        | true, true, Bool(false), Bool(true) -> ...
        | true, true, _, _ -> raise ( RuntimeError "Both guards are false")
        | _, _, _, _ -> raise ( RuntimeError "Wrong type")
        )
        *)
    (* ForEach 
    | ForEach(elist, efun) -> (* elist : exp list *)
        let vlist = List.map (fun e -> eval e s) elist in 
        if (* controllo che tutti i vlist siano Int *) then
            let fclosure = eval efun s in
            (match fclosure with
            | Closure(fx, fbody, fenv) ->
                let newvlist = List.map (fun v ->
                    let newenv = bind fenv fx v in
                    eval fbody newenv
                ) vlist in
                Int (List.fold_left (fun v acc -> match v with
                    | Int(n) -> acc + n
                    | _ -> failwith("Type error") ) 0 newvlist)
            )
        else
            failwith "Type error"
    *)

val eval : exp -> evT env -> evT = <fun>


In [6]:
let test1 = On(
    CstFalse, EInt 10,
    CstFalse, EInt 4
);;

eval test1 emptyenv;;

val test1 : exp = On (CstFalse, EInt 10, CstFalse, EInt 4)


error: runtime_error

In [7]:
let test2 = On(
    EInt 3, EInt 10,
    CstFalse, EInt 4
);;

eval test2 emptyenv;;

val test2 : exp = On (EInt 3, EInt 10, CstFalse, EInt 4)


error: runtime_error

## Esercizio

Definire un interprete per il linguaggio MiniPixel
```
Pixel             p ::= < r, g, b >      dove r, g, b ∈ {0, 1, . . . , 255}
Identificatori    I ::= . . .
Espressioni       e ::= I | p | lighten e | darken e | let I = e1 in e2
```

In [8]:
type ide = string;;

(* Sintassi del nostro linguaggio: espressioni *)
type exp =
    | Den of ide
    | EPixel of int * int * int
    | Lighten of exp
    | Darken of exp
    | Let of ide * exp * exp;;

type 't env = ide -> 't;;

(* Valori del linguaggio *)
type evT =
    | Pixel of int * int * int
    | UnBound;;

let emptyenv = fun x -> UnBound;;

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

let rec eval (e: exp) (s: evT env) : evT =
    match e with
    | Den(x) -> s x
    | EPixel(r, g, b) -> Pixel(r, g, b)
    | Lighten(e) ->
        let v: evT = eval e s in
        (match v with
        | Pixel(r, g, b) -> Pixel(r+1, g+1, b+1)
        | _ -> failwith "Errore: UnBound"
        )
    | Darken(e) ->
        let v = eval e s in
        (match v with
        | Pixel(r, g, b) -> Pixel(r-1, g-1, b-1)
        | _ -> failwith "Errore: UnBound"
        )
    | Let(x, e1, ebody) ->
        let v: evT = eval e1 s in
        let newenv = bind s x v in
        eval ebody newenv;;

type ide = string


type exp =
    Den of ide
  | EPixel of int * int * int
  | Lighten of exp
  | Darken of exp
  | Let of ide * exp * exp


type 't env = ide -> 't


type evT = Pixel of int * int * int | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


val eval : exp -> evT env -> evT = <fun>


In [9]:
let incr x = if x = 255 then 255 else x + 1;;
let decr x = if x = 0 then 0 else x - 1;;

let rec eval (e: exp) (s: evT env) : evT =
    match e with
    | Den(x) -> s x
    | EPixel(r, g, b) -> 
        if 0 <= r && r <= 255 && 0 <= g && g <= 255 && 0 <= b && b <= 255 then
            Pixel(r, g, b)
        else
            failwith "Invalid constant"
    | Lighten(e) ->
        let v: evT = eval e s in
        (match v with
        | Pixel(r, g, b) -> Pixel(incr r, incr g, incr b)
        | _ -> failwith "Errore: UnBound"
        )
    | Darken(e) ->
        let v = eval e s in
        (match v with
        | Pixel(r, g, b) -> Pixel(decr r, decr g, decr b)
        | _ -> failwith "Errore: UnBound"
        )
    | Let(x, e1, ebody) ->
        let v: evT = eval e1 s in
        let newenv = bind s x v in
        eval ebody newenv;;

val incr : int -> int = <fun>


val decr : int -> int = <fun>


val eval : exp -> evT env -> evT = <fun>


In [10]:
let test1 = Let (
    "x",
    Darken (EPixel (255, 0, 128)),
    Lighten (Den "x")
);;

eval test1 emptyenv;;

val test1 : exp = Let ("x", Darken (EPixel (255, 0, 128)), Lighten (Den "x"))


- : evT = Pixel (255, 1, 128)
